In [1]:
import pickle
from pathlib import Path
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import cv2
import numpy as np
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
def pick_first_three(loaded_list):
    new_list = []
    for item in loaded_list:
        try:
            number = int(item[:3])
            new_list.append(number)
        except:
            new_list.append(-1)
    return new_list

In [101]:
with open('zip_lists_9968.pkl', 'rb') as f:
    zip_load_list = pickle.load(f)

with open('image_lists_9968.pkl', 'rb') as f:
    image_load_list = pickle.load(f)

In [102]:
zip_filt_list = pick_first_three(zip_load_list)

In [103]:
a = Counter(zip_filt_list) #482 
#a

#9: 941, 945 940 
#4: 482, 481 480 483

In [107]:
f_zip_list = []
f_images_list = []
for i in range(len(zip_filt_list)):
    #print(zip_filt_list[i])
    if zip_filt_list[i] == 941:
        f_zip_list.append(9)
        f_images_list.append(image_load_list[i])
    if zip_filt_list[i] == 482:
        f_zip_list.append(4)
        f_images_list.append(image_load_list[i])

In [122]:
test_zip_list = []
test_images_list = []
for i in range(len(zip_filt_list)):
    #print(zip_filt_list[i])
    if zip_filt_list[i] == 945 or zip_filt_list[i] == 940:
        test_zip_list.append(9)
        test_images_list.append(image_load_list[i])
    if zip_filt_list[i] == 481 or zip_filt_list[i] == 480 or zip_filt_list[i] == 483:
        test_zip_list.append(4)
        test_images_list.append(image_load_list[i])

In [ ]:
unique_values = sorted(set(test_zip_list))  # 獲得唯一值並排序
mapping = {value: index for index, value in enumerate(unique_values)}  # 創建映射字典

# 使用映射字典來轉換原始列表
test_zip_list = [mapping[value] for value in test_zip_list]

# CA filter

In [106]:
f_zip_list = []
f_images_list = []

for i in range(len(zip_filt_list)):
    if(str(zip_filt_list[i])[0] == '4') and a[zip_filt_list[i]] < 800 and a[zip_filt_list[i]] > 90:
        f_zip_list.append(zip_filt_list[i])
        f_images_list.append(image_load_list[i])

In [108]:
unique_values = sorted(set(f_zip_list))  # 獲得唯一值並排序
mapping = {value: index for index, value in enumerate(unique_values)}  # 創建映射字典

# 使用映射字典來轉換原始列表
f_zip_list = [mapping[value] for value in f_zip_list]

In [116]:
X = np.array(f_images_list) 
X = X.reshape((-1, 256, 256, 1)) 
y = to_categorical(f_zip_list, num_classes=len(set(f_zip_list)))  

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [109]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [117]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(set(f_zip_list)), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [118]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

def build_model(input_shape=(256, 256, 1), num_classes=len(set(f_zip_list))):
    # 加載ResNet50模型，不包括頂層，預設輸入尺寸為(224, 224, 3)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # 凍結基模型的權重
    base_model.trainable = False

    # 建立新模型
    model = models.Sequential([
        # 因為ResNet50預設輸入尺寸為(224, 224, 3)，所以需要調整輸入圖像尺寸和通道
        layers.experimental.preprocessing.Resizing(224, 224),
        layers.Conv2D(3, (3, 3), padding='same'),  # 將圖像從1通道轉為3通道
        base_model,         
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

model_ResNet50 = build_model()
model_ResNet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [119]:
print(len(set(f_zip_list)),len(f_zip_list))

2 2217


In [123]:
model_ResNet50.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val))

test_loss, test_acc = model_ResNet50.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

22/49 [============>.................] - ETA: 1:02 - loss: 0.3779 - accuracy: 0.8409

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")